## Toxic Comment Classification THE SCRAPPING

## PART I

In [14]:
get_username = 'USERNAME'
get_pass = 'PASSWORD'
get_topic = 'ENTER SEARCH TOPIC'

In [9]:
!pip install selenium

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import re
from getpass import getpass
from time import sleep
import csv

In [3]:
driver = Chrome()

In [4]:
driver.get('https://www.twitter.com/login')
driver.maximize_window()
#change sleep values according to internet speed
sleep(5)

In [9]:
username = driver.find_element("xpath",'//input[@name="text"]')
username.send_keys(get_username)
username.send_keys(Keys.RETURN)
sleep(3)
password=driver.find_element("xpath",'//input[@name="password"]')
password.send_keys(get_pass)
password.send_keys(Keys.RETURN)
sleep(10)

In [11]:
#check for any popup by twitter
search_input = driver.find_element("xpath",'//input[@aria-label="Search query"]')
search_input.send_keys(get_topic)
search_input.send_keys(Keys.ENTER)
sleep(1)
driver.find_element("link text",'Latest').click()

In [12]:
def get_tweet_data(card):
    """Extract data from tweet card"""

    try:
        handle = card.find_element("xpath",'.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element("xpath",'.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element("xpath",'.//div[2]/div[2]/div[1]').text
    responding = card.find_element("xpath",'.//div[2]/div[2]/div[2]').text
    text = comment + responding

    tweet = (handle, text)
    
    return tweet

In [13]:
#get all tweets on the page
data=[]
tweet_ids= set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements("xpath",'//article[@data-testid="tweet"]')
    for card in page_cards[-30:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)


#ENTER A SAVE FILE NAME FOR SAVING THE .csv FILE
#RECOMMENDED TO USE NAME SAME AS TOPIC
        with open('.csv', 'w', newline='', encoding='utf-8') as f:
            header = ['Handle', 'Text']
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerows(data)

            
    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            with open('#ENTER SAME NAME AS ABOVE SAVE.csv', 'w', newline='', encoding='utf-8') as f:
                header = ['Handle', 'Text']
                writer = csv.writer(f)
                writer.writerow(header)
                writer.writerows(data)
            break
                    
driver.close()

KeyboardInterrupt: 